# Ledoit–Wolf Shrinkage Covariance


This notebook computes Ledoit-Wolf Shrinkage covariance. This method is particularly useful when:
1. Number of asset < Number of time observations
2. High dimensional portfolios
3. Whenever the covariance matrix is singular (i.e., not invertible)

The Ledoit–Wolf (LW) methodology corrects the instability of the sample covariance by shrinking it toward a structured target. The process involves several key steps:

1. **Compute the sample covariance**  
   Given mean-centered returns $X \in \mathbb{R}^{T \times N}$, calculate the sample covariance  
   
   $$
   S = \frac{1}{T} X^\top X
   $$ 
   
   which may be ill-conditioned when $T$ is small.

2. **Define the shrinkage target**  
   The target matrix $F = \mu I$ represents a simplified structure assuming all assets have the same variance $\mu = \frac{1}{N}\text{tr}(S) $ and are uncorrelated. This serves as prior toward which the noisy sample covariance is shrinked.

3. **Measure the noise in the sample covariance**  
   Estimate the expected squared deviation of each sample outer product $( x_t x_t^\top )$ from $S$:  
   
   $$
   \widehat{\phi} = \frac{1}{T^2}\sum_{t=1}^{T} \|x_t x_t^\top - S\|_F^2
   $$
   
   This term quantifies the sampling variability (noise) in S.

4. **Compute the shrinkage intensity**  
   The optimal shrinkage parameter  
   
   $$
   \alpha = \frac{\widehat{\phi}}{\|S - F\|_F^2}
   $$  
   
   determines how much weight is placed on the structured target. Intuitively, is the noise-distance ratio. It lies between 0 (no shrinkage) and 1 (full shrinkage to $F$ ).

5. **Form the shrunk covariance matrix**  
   Combine the two components to obtain the LW estimator:  
   
   $$
   \widehat{\Sigma} = (1 - \alpha) S + \alpha F
   $$
   
   This yields a positive definite, better-conditioned covariance estimate.

6. **Interpretation**  
   The intuition is to trade off bias and variance: the sample covariance is unbiased but high-variance, while the target is biased but low-variance. The LW estimator automatically balances these properties to minimize mean-squared error, leading to improved stability and invertibility for portfolio optimization and risk modeling.

## 1. Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.covariance import LedoitWolf

plt.close('all')

## 2. Implementation

In [2]:
def ledoit_wolf_shrinkage_corrected(X):
    X = np.asarray(X, dtype=float)
    n, p = X.shape
    Xc = X - X.mean(axis=0, keepdims=True)
    S = (Xc.T @ Xc) / n
    mu = np.trace(S) / p
    F = mu * np.eye(p)
    
    delta2 = np.sum((S - F)**2)

    acc = 0.0
    for t in range(n):
        xt = Xc[t:t+1, :]
        diff = (xt.T @ xt) - S
        acc += np.sum(diff * diff)
    phi = acc / (n**2)
    alpha = float(np.clip(phi / delta2, 0.0, 1.0))
    Sigma_hat = (1 - alpha) * S + alpha * F
    return Sigma_hat, alpha

## 3. Compute Ledoit-Wolf

In [3]:
Ret = pd.read_excel('../Data/Returns_example.xlsx').iloc[:, 1:]
Sigma = np.cov(Ret.T)

try:
    print(np.linalg.inv(Sigma))

except np.linalg.LinAlgError:
    print("Sigma is not invertible!")
    

Sigma_hat, alpha = ledoit_wolf_shrinkage_corrected(Ret)
print("Shrinkage factor: ", alpha)

Sigma is not invertible!
Shrinkage factor:  0.047233083951648755


## 4. Verify against scikit-learn

In [51]:

lw = LedoitWolf().fit(Ret - Ret.mean())

Sigma_lw = lw.covariance_

alpha_lw = lw.shrinkage_

rel_err = np.linalg.norm(Sigma_hat - Sigma_lw, 'fro') / np.linalg.norm(Sigma_lw, 'fro')

print({'alpha_ours': alpha, 'alpha_sklearn': alpha_lw, 'rel_fro_error': rel_err})


{'alpha_ours': 0.047233083951648755, 'alpha_sklearn': 0.04723308395164878, 'rel_fro_error': 1.522636899416092e-16}
